In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import commonmodules as cm

def compute_and_plot (df, vib_values , temp_values, testsetvib, \
   minvalue, maxvalue, ofp):

  maxt = max(temp_values)
  mint = min(temp_values)

  minv = min(vib_values)
  maxv = max(vib_values)

  cm.plotfull3dcurve (df, vib_values, temp_values)

  train_xy, train_z, test_xy, test_z = cm.get_train_and_test_rmv (temp_values, vib_values, \
    df, testsetvib)

  fig = plt.figure()
  ax = fig.add_subplot(projection='3d')

  for i in range(train_z.shape[0]):
    x = train_xy[i,0]
    y = train_xy[i,1]
    z = train_z[i]
    ax.scatter(x, y, z, marker="o", color="g")

  for i in range(test_z.shape[0]):
    x = test_xy[i,0]
    y = test_xy[i,1]
    z = test_z[i]
    ax.scatter(x, y, z, marker="o", color="r")

  ax.set_xlabel('X Label')
  ax.set_ylabel('Y Label')
  ax.set_zlabel('Z Label')
  plt.gcf().set_size_inches(20, 15)

  plt.show()

  epochs = 50
  batch_size = 50

  model = cm.build_model_NN_1()
  history = model.fit(train_xy, train_z, epochs=epochs,  batch_size=batch_size, \
    verbose=1)

  z_pred = model.predict(train_xy)
  trainmse = 0.0
  denorm_trainmse = 0.0
  cont = 0.0
  
  for i in range(train_z.shape[0]):
    x = train_xy[i,0]
    y = train_xy[i,1]

    z = train_z[i]
    denorm_z = (z * (maxvalue - minvalue))+minvalue

    zpred = z_pred[i]
    denorm_zpred = (zpred * (maxvalue - minvalue))+minvalue
    
    trainmse += (zpred-z)**2
    denorm_trainmse += (denorm_zpred-denorm_z)**2

    cont += 1.0

    #print("Train, %10.7f , %10.7f , %10.7f , %10.7f , %10.7f"%(z, y, z, zpred, zstd))

  trainmse = trainmse/cont
  denorm_trainmse = denorm_trainmse/cont

  z_pred = model.predict(test_xy)
  mse = 0.0
  denorm_mse = 0.0
  cont = 0.0
  for i in range(test_z.shape[0]):

    z = test_z[i]
    denorm_z = (z * (maxvalue - minvalue))+minvalue

    zpred = z_pred[i]
    denorm_zpred = (zpred * (maxvalue - minvalue))+minvalue

    mse += (zpred-z)**2
    denorm_mse = (denorm_zpred-denorm_z)**2

    cont += 1.0

  mse = mse/cont
  denorm_mse = denorm_mse/cont

  print("Normalized TestSet MSE , ", mse[0], " , TrainSet MSE ,", trainmse[0], \
    flush=True, file=ofp)
  print("TestSet MSE , ", denorm_mse[0], " , TrainSet MSE ,", denorm_trainmse[0], \
    flush=True, file=ofp)

  for vslct in testsetvib:
    
    print("V = ", vslct)
    print("T , V, True Value , Predicted Value , Denorm. True Value , Denorm. Predicted Value", \
      file=ofp)

    for i in range(test_z.shape[0]):
      t = test_xy[i,0]
      t = int(t*(maxt - mint)+mint)
      v = test_xy[i,1]
      v = int(v*(maxv - minv)+minv)
      if (v == vslct):

        z = test_z[i]
        zpred = z_pred[i]
        
        denorm_z = (z*(maxvalue - minvalue))+minvalue
        denorm_zpred = (zpred*(maxvalue - minvalue))+minvalue
    
        print("%10.2f , %10.2f , %10.7e , %10.7e , %10.7e , %10.7e "%(\
          t, v, z, zpred, denorm_z, denorm_zpred), file=ofp)


In [ ]:
filename = "dv1.xlsx"
df, vib_values , temp_values, minvalue, maxvalue = cm.filterinitialset (filename)
testsetvib = [34, 36]
ofp = open("NN1_dv1.csv", "w")
compute_and_plot (df, vib_values , temp_values, testsetvib, minvalue, maxvalue, ofp)
ofp.close()

In [ ]:
filename = "dv2.xlsx"
df, vib_values , temp_values, minvalue, maxvalue = cm.filterinitialset (filename)
testsetvib = [34, 36]
ofp = open("NN1_dv2.csv", "w")
compute_and_plot (df, vib_values , temp_values, testsetvib, minvalue, maxvalue, ofp)
ofp.close()

In [ ]:
filename = "dv3.xlsx"
df, vib_values , temp_values, minvalue, maxvalue = cm.filterinitialset (filename)
testsetvib = [34, 36]
ofp = open("NN1_dv3.csv", "w")
compute_and_plot (df, vib_values , temp_values, testsetvib, minvalue, maxvalue, ofp)
ofp.close()

In [ ]:
import tensorflow as tf

nn1 = cm.build_model_NN_1()
tf.keras.utils.plot_model(nn1, to_file="nn1.png", show_shapes=True)
nn2 = cm.build_model_NN_2()
tf.keras.utils.plot_model(nn2, to_file="nn2.png", show_shapes=True)
nn3 = cm.build_model_NN_3()
tf.keras.utils.plot_model(nn3, to_file="nn3.png", show_shapes=True)